<a href="https://colab.research.google.com/github/mehmetalivarol/deep_learning/blob/master/borusan_ses.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from IPython import get_ipython
get_ipython().magic('reset -sf')

In [2]:
pip install pydub

In [3]:
#gerekli kütüphaneler
import pandas as pd
import os
import IPython.display as ipd
import numpy as np
import librosa
import librosa.display
import matplotlib.pyplot as plt
import string
from pydub.utils import mediainfo
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, Conv2D, MaxPooling2D, GlobalAveragePooling2D
from keras.optimizers import Adam
from keras.utils import np_utils
from sklearn import metrics 


Using TensorFlow backend.


In [4]:
#örnek ses dosyası
sound_path = '/content/drive/My Drive/Colab Data/borusan/ses/'
ipd.Audio(sound_path+'000000000077000532_HEALTHY_1200RPM_7ecd206d-3955-451f-9821-07a09d5ceeb0.mp3')

In [0]:
df=[]
for i in os.listdir(sound_path):
    if i.endswith('.mp3'):
      df.append(i)


In [0]:
df=pd.DataFrame(df)
df=df.rename(columns={0:"name"})
df["name"] = df["name"].str.replace("FUEL_SYSTEM","FUELSYSTEM",case = False)
df["name"] = df["name"].str.replace("HANDLE_BEDS","HANDLEBEDS",case = False)

In [0]:
df[['id','status','rpm','code']] = df.name.apply(lambda x: pd.Series(str(x).split("_")))
df["name"] = df["name"].str.replace("FUELSYSTEM","FUEL_SYSTEM",case = False)
df["name"] = df["name"].str.replace("HANDLEBEDS","HANDLE_BEDS",case = False)

In [8]:
df.head()

,name,id,status,rpm,code
0,JEX10008_HEALTHY_1000RPM_917c17da-9289-4960-a5...,JEX10008,HEALTHY,1000RPM,917c17da-9289-4960-a50d-36567c609189.mp3
1,LYK00446_HEALTHY_2100RPM_07e15799-3f22-4d2e-b8...,LYK00446,HEALTHY,2100RPM,07e15799-3f22-4d2e-b808-5f9553f8afda.mp3
2,LXX00217_HEALTHY_700RPM_69f6df6f-c6af-4de7-983...,LXX00217,HEALTHY,700RPM,69f6df6f-c6af-4de7-9831-e72a6743e67a.mp3
3,000000000000042615_HANDLE_BEDS_750RPM_1822588e...,000000000000042615,HANDLEBEDS,750RPM,1822588e-c604-4165-91ab-83fea8636d2f.mp3
4,000000000077000532_HEALTHY_1200RPM_7ecd206d-39...,000000000077000532,HEALTHY,1200RPM,7ecd206d-3955-451f-9821-07a09d5ceeb0.mp3


In [9]:
print(df.status.value_counts())

HEALTHY       93
FUELSYSTEM    18
OTHER         17
HANDLEBEDS     4
VALVES         3
TURBO          2
Name: status, dtype: int64


Handle_Beds,Valves ve Turbo sınıflarındaki gözlemler çok az!!! Yapabiliyorsak bu kısımlar için bir örnek almak gerekir.
Other ise bir diğer sorun!!Tanımlanmamış bir arıza mı yoksa sağlıklı bir durum mu?

In [0]:
#healthy status figure
file = (sound_path+'/000000000077000532_HEALTHY_1200RPM_7ecd206d-3955-451f-9821-07a09d5ceeb0.mp3')
plt.figure(figsize=(16,9))
data,rate = librosa.load(file)
tmp = librosa.display.waveplot(data,sr=rate)

In [0]:
#fuel_system status figure
file = (sound_path+'/4ND00110_FUEL_SYSTEM_800RPM_9668ae15-76ca-4d24-99cc-fcfef2191992.mp3')
plt.figure(figsize=(16,9))
data,rate = librosa.load(file)
tmp = librosa.display.waveplot(data,sr=rate)

In [0]:
#handle_beds status figure
file = (sound_path+'/000000000000042615_HANDLE_BEDS_750RPM_1822588e-c604-4165-91ab-83fea8636d2f.mp3')
plt.figure(figsize=(16,9))
data,rate = librosa.load(file)
tmp = librosa.display.waveplot(data,sr=rate)

In [0]:
#valves status figure
file = (sound_path+'/BBD02736_VALVES_555RPM_9c2517eb-9fe8-450d-a5d0-83a7a3e6cd6f.mp3')
plt.figure(figsize=(16,9))
data,rate = librosa.load(file)
tmp = librosa.display.waveplot(data,sr=rate)

In [0]:
#turbo status figure
file = (sound_path+'/LCS01409_TURBO_950RPM_85c50c42-0203-4654-96f0-66d5903242a7.mp3')
plt.figure(figsize=(16,9))
data,rate = librosa.load(file)
tmp = librosa.display.waveplot(data,sr=rate)

In [0]:
#other status figure
file = (sound_path+'/JTF23363_OTHER_1250RPM_2163923b-d635-43b3-819d-7c09a4a66616.mp3')
plt.figure(figsize=(16,9))
data,rate = librosa.load(file)
tmp = librosa.display.waveplot(data,sr=rate)

In [0]:
audiodata = []
for index,row in df.iterrows():
    filename = os.path.join(os.path.abspath(sound_path),str(row["name"]))
    data = mediainfo(filename)
    audiodata.append(data)

In [0]:
audiodata

In [0]:
# Convert into a Panda dataframe
audiodf = pd.DataFrame(audiodata, columns=['filename','bit_rate','channel_layout','channels','duration','sample_fmt','sample_rate','size'])
audiodf["filename"] = audiodf["filename"].str.replace(sound_path,"",case = False)

In [27]:
audiodf["name"] = audiodf["filename"].str.replace("FUEL_SYSTEM","FUELSYSTEM",case = False)
audiodf["name"] = audiodf["filename"].str.replace("HANDLE_BEDS","HANDLEBEDS",case = False)
audiodf["name"] = audiodf["name"].str.replace(".mp3","",case = False)
audiodf[['id','status','rpm','code','nan']] = audiodf.name.apply(lambda x: pd.Series(str(x).split("_")))
audiodf["status"] = audiodf["status"].str.replace("FUELSYSTEM","FUEL_SYSTEM",case = False)
audiodf["status"] = audiodf["status"].str.replace("HANDLEBEDS","HANDLE_BEDS",case = False)
audiodf.drop(columns=['nan'])

,filename,bit_rate,channel_layout,channels,duration,sample_fmt,sample_rate,size,name,id,status,rpm,code
0,JEX10008_HEALTHY_1000RPM_917c17da-9289-4960-a5...,128031,stereo,2,63.268571,s16p,44100,1012550,JEX10008_HEALTHY_1000RPM_917c17da-9289-4960-a5...,JEX10008,HEALTHY,1000RPM,917c17da-9289-4960-a50d-36567c609189
1,LYK00446_HEALTHY_2100RPM_07e15799-3f22-4d2e-b8...,128023,stereo,2,86.256327,s16p,44100,1380354,LYK00446_HEALTHY_2100RPM_07e15799-3f22-4d2e-b8...,LYK00446,HEALTHY,2100RPM,07e15799-3f22-4d2e-b808-5f9553f8afda
2,LXX00217_HEALTHY_700RPM_69f6df6f-c6af-4de7-983...,128033,stereo,2,59.977143,s16p,44100,959887,LXX00217_HEALTHY_700RPM_69f6df6f-c6af-4de7-983...,LXX00217,HEALTHY,700RPM,69f6df6f-c6af-4de7-9831-e72a6743e67a
3,000000000000042615_HANDLE_BEDS_750RPM_1822588e...,128530,stereo,2,3.813878,s16p,44100,61275,000000000000042615_HANDLEBEDS_750RPM_1822588e-...,000000000000042615,HANDLE_BEDS,750RPM,1822588e-c604-4165-91ab-83fea8636d2f
4,000000000077000532_HEALTHY_1200RPM_7ecd206d-39...,128045,stereo,2,44.199184,s16p,44100,707439,000000000077000532_HEALTHY_1200RPM_7ecd206d-39...,000000000077000532,HEALTHY,1200RPM,7ecd206d-3955-451f-9821-07a09d5ceeb0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
132,SWL04260_HEALTHY_800RPM_d6a0c803-63db-4874-a82...,128033,stereo,2,60.943673,s16p,44100,975351,SWL04260_HEALTHY_800RPM_d6a0c803-63db-4874-a82...,SWL04260,HEALTHY,800RPM,d6a0c803-63db-4874-a829-8ba102aa1e14
133,TYE00189_HANDLE_BEDS_800RPM_86590af1-32d4-494d...,128058,stereo,2,34.586122,s16p,44100,553630,TYE00189_HANDLEBEDS_800RPM_86590af1-32d4-494d-...,TYE00189,HANDLE_BEDS,800RPM,86590af1-32d4-494d-bf06-a2b1b3257287
134,JFW00743_HEALTHY_750RPM_eb4e974a-f518-4ba3-a87...,128033,stereo,2,60.473469,s16p,44100,967828,JFW00743_HEALTHY_750RPM_eb4e974a-f518-4ba3-a87...,JFW00743,HEALTHY,750RPM,eb4e974a-f518-4ba3-a878-88eb1391aa0b
135,N8F01338_HEALTHY_800RPM_0d9ff240-737a-409d-a73...,128041,stereo,2,48.796735,s16p,44100,781000,N8F01338_HEALTHY_800RPM_0d9ff240-737a-409d-a73...,N8F01338,HEALTHY,800RPM,0d9ff240-737a-409d-a734-5f19c656a1da


In [0]:
import numpy as np
max_pad_len = 4000
def extract_features(file_name):
    try:
        audio, sample_rate = librosa.load(file_name, res_type='kaiser_fast') 
        mfccs = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40)
        pad_width = max_pad_len - mfccs.shape[1]
        mfccs = np.pad(mfccs, pad_width=((0, 0), (0, pad_width)), mode='constant')
    except Exception as e:
        print("Error encountered while parsing file: ", file_name)
        return None    
    return mfccs

In [37]:
# Load various imports 
import pandas as pd
import os
import librosa

features = []

# Iterate through each sound file and extract the features 
for index, row in audiodf.iterrows():
    
    file_name = os.path.join(os.path.abspath(sound_path),str(row["filename"]))
    
    class_label = row["status"]
    data = extract_features(file_name)
    
    features.append([data, class_label])

# Convert into a Panda dataframe 
featuresdf = pd.DataFrame(features, columns=['feature','class_label'])

print('Finished feature extraction from ', len(featuresdf), ' files')

Finished feature extraction from  137  files


In [0]:
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical

# Convert features and corresponding classification labels into numpy arrays
X = np.array(featuresdf.feature.tolist())
y = np.array(featuresdf.class_label.tolist())

# Encode the classification labels
le = LabelEncoder()
yy = to_categorical(le.fit_transform(y)) 

# split the dataset 
from sklearn.model_selection import train_test_split 

x_train, x_test, y_train, y_test = train_test_split(X, yy, test_size=0.2, random_state = 42)

In [0]:
num_rows = 40
num_columns = 4000
num_channels = 1

x_train = x_train.reshape(x_train.shape[0], num_rows, num_columns, num_channels)
x_test = x_test.reshape(x_test.shape[0], num_rows, num_columns, num_channels)

num_labels = yy.shape[1]
filter_size = 2

# Construct model 
model = Sequential()
model.add(Conv2D(filters=16, kernel_size=2, input_shape=(num_rows, num_columns, num_channels), activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.2))

model.add(Conv2D(filters=32, kernel_size=2, activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.2))

model.add(Conv2D(filters=64, kernel_size=2, activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.2))

model.add(Conv2D(filters=128, kernel_size=2, activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.2))
model.add(GlobalAveragePooling2D())

model.add(Dense(num_labels, activation='softmax'))

In [41]:
# Compile the model
model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam')

In [42]:
# Display model architecture summary 
model.summary()

# Calculate pre-training accuracy 
score = model.evaluate(x_test, y_test, verbose=1)
accuracy = 100*score[1]

print("Pre-training accuracy: %.4f%%" % accuracy)

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5 (Conv2D)            (None, 39, 3999, 16)      80        
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 19, 1999, 16)      0         
_________________________________________________________________
dropout_5 (Dropout)          (None, 19, 1999, 16)      0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 18, 1998, 32)      2080      
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 9, 999, 32)        0         
_________________________________________________________________
dropout_6 (Dropout)          (None, 9, 999, 32)        0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 8, 998, 64)       

In [43]:
from keras.callbacks import ModelCheckpoint 
from datetime import datetime 

#num_epochs = 12
#num_batch_size = 128

num_epochs = 72
num_batch_size = 256

checkpointer = ModelCheckpoint(filepath='/content/drive/My Drive/Colab Data/borusan/ses/saved_models/weights.best.basic_cnn.hdf5', 
                               verbose=1, save_best_only=True)
start = datetime.now()

model.fit(x_train, y_train, batch_size=num_batch_size, epochs=num_epochs, validation_data=(x_test, y_test), callbacks=[checkpointer], verbose=1)

duration = datetime.now() - start
print("Training completed in time: ", duration)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 109 samples, validate on 28 samples
Epoch 1/72
109/109 [==============================] - 14s 127ms/step - loss: 4.0026 - acc: 0.0092 - val_loss: 1.3250 - val_acc: 0.5000

Epoch 00001: val_loss improved from inf to 1.32501, saving model to /content/drive/My Drive/Colab Data/borusan/ses/saved_models/weights.best.basic_cnn.hdf5


OSError: ignored